In [231]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.autograd import Variable
import numpy as np
import random
from PIL import Image
from ipywidgets import FloatProgress
from IPython.display import display

In [178]:
def readImages(imageFile="imList.txt", openAll=True):
    """
        args :
            imageFile = file with one image path per line
            openAll = bool : load images in memory or not
        ret :
            with openAll : <image path list>, <image list>
            without openAll : <image path list>
    """
    with open(imageFile) as f:
        imList = f.read().splitlines()
        if openAll:
            imOpen = []
            for im in imList:
                i = Image.open(im).resize((225, 225), Image.BILINEAR)
                if openAll:
                    imOpen.append(i)
            return imList, imOpen
        else:
            return imList

In [203]:
def ComputeMean(imagesList):
    """
        TODO : make efficient
    """
    r,g,b = 0,0,0
    toT = transforms.ToTensor()
    h = len(imagesList[0])
    w = len(imagesList[0][0])

    #f = FloatProgress(min=0, max=len(imagesList))
    #display(f)

    for im in imagesList:
        #f.value += 1
        t = toT(im)
        for e in t[0].view(-1):
            r += e
        for e in t[1].view(-1):
            g += e
        for e in t[2].view(-1):
            b += e
    return r(len(imagesList)*h*w), g/(len(imagesList)*h*w), b/(len(imagesList)*h*w) 

In [214]:
def ComputeStdDev(imagesList, mean):
    """
        TODO : make efficient
    """
    toT = transforms.ToTensor()
    r,g,b = 0,0,0
    h = len(toT(imagesList[0])[0])
    w = len(toT(imagesList[0])[0][0])
    for im in imagesList:
        i = toT(im)
        for e in t[0].view(-1):
            r += (e - mean[0])**2
        for e in t[1].view(-1):
            g += (e - mean[1])**2
        for e in t[2].view(-1):
            b += (e - mean[2])**2
    return (r/(len(imagesList)*h*w))**0.5, (g/(len(imagesList)*h*w))**0.5, (b/(len(imagesList)*h*w))**0.5

Read the dataset and compute the mean and std dev :

In [215]:
trainset, imagesList = readImages("CliList.txt")
m = ComputeMean(imagesList)
print(m)
s = ComputeStdDev(imagesList, m)
print(s)

(0.42602490885018174, 0.4269285229908378, 0.418182238544934)
(0.20014586928330125, 0.17607878531703874, 0.17040668227814146)


Define the network as class (from nn.Module) :

In [217]:
class maxnet(nn.Module):
    def __init__(self, nbClass=464):
        super(maxnet, self).__init__()
        self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1)),
                nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1)),
                nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000),
            nn.ReLU(True),
            nn.Linear(1000, nbClass),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

Training

In [291]:
mymodel = maxnet()

criterion = nn.loss.CrossEntropyLoss()
optimizer = optim.SGD(mymodel.parameters(), lr=0.01, momentum=0.9)

#trainset, imagesList = readImages("CliList.txt")
#testset, imagesTest = readImages("CliListTest.txt")
labels = open("CliConcept.txt").read().splitlines()

toT = transforms.ToTensor()

batchSize = 32

for epoch in range(2): # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(len(trainset)/batchSize):
        # get the inputs
        elIndex = [random.randrange(0, len(trainset)) for j in range(batchSize)]
        
        inputs = torch.Tensor(batchSize,3,225,225)
        for j in range(batchSize):
            inputs[j] = toT(imagesList[elIndex[j]])
        inputs = Variable(inputs)
        lab = Variable(torch.LongTensor([labels.index(trainset[j].split('/')[-1].split('-')[0]) for j in elIndex]))
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = mymodel(inputs)
        loss = criterion(outputs, lab)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        if i % 10 == 9: # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0
        #if i % 20 == 19:#test every 20 mini-batches
        if i % 20 == 19:
            print('test :')
            correct = 0
            tot = 0
            for j in range(len(testset)):
                inp = torch.Tensor(1,3,225,225)
                inp[0] = toT(imagesTest[j])
                outputs = mymodel(Variable(inp))
                _, predicted = torch.max(outputs.data, 1)
                predicted = predicted.tolist()[0][0]
                correct += (predicted == labels.index(testset[i].split('/')[-1].split('-')[0]))
                tot += 1
            print("Correct : ", correct, "/", tot)
                
            
print('Finished Training')

[1,    10] loss: 6.139
[1,    20] loss: 6.140
test :
('Correct : ', 0, '/', 177)
[1,    30] loss: 6.140
[1,    40] loss: 6.139
test :
('Correct : ', 0, '/', 177)
[1,    50] loss: 6.138
[1,    60] loss: 6.139
test :
('Correct : ', 0, '/', 177)
[1,    70] loss: 6.139
[1,    80] loss: 6.135
test :
('Correct : ', 0, '/', 177)
[1,    90] loss: 6.134
[1,   100] loss: 6.134
test :
('Correct : ', 0, '/', 177)
[2,    10] loss: 6.134
[2,    20] loss: 6.136
test :
('Correct : ', 0, '/', 177)
[2,    30] loss: 6.134
[2,    40] loss: 6.133
test :
('Correct : ', 0, '/', 177)
[2,    50] loss: 6.134
[2,    60] loss: 6.129
test :
('Correct : ', 0, '/', 177)
[2,    70] loss: 6.131
[2,    80] loss: 6.129
test :
('Correct : ', 0, '/', 177)
[2,    90] loss: 6.133
[2,   100] loss: 6.129
test :
('Correct : ', 0, '/', 177)
Finished Training


In [287]:
t = torch.LongTensor([240])
t.tolist()

[240]